### TAO remote client - Classification

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)

### Sample prediction for an Image Classification model
<img align="center" src="../example_images/sample_image_classification.jpg">

### The workflow in a nutshell

- Creating a dataset
- Upload dataset to the service
- Getting a PTM from NGC
- Model Actions
    - Train (Normal/AutoML)
    - Evaluate
    - Prune, retrain
    - Export
    - Tao-Deploy
    - Inference on TAO

### Table of contents

1. [Install TAO remote client ](#head-1)
1. [Set the remote service base URL](#head-2)
1. [Access the shared volume](#head-3)
1. [Create the datasets](#head-4)
1. [List datasets](#head-5)
1. [Create a model experiment](#head-8)
1. [Find pretrained model](#head-9)
1. [Customize model metadata](#head-10)
1. [View hyperparameters that are enabled for AutoML by default](#head-11)
1. [Set AutoML related configurations](#head-12)
1. [Provide train specs](#head-13)
1. [Run train](#head-14)
1. [View checkpoint files](#head-15)
1. [Provide evaluate specs](#head-16)
1. [Run evaluate](#head-17)
1. [Provide prune specs](#head-18)
1. [Run prune](#head-19)
1. [Provide retrain specs](#head-20)
1. [Run retrain](#head-21)
1. [Run evaluate on retrain](#head-21-1)
1. [Provide export specs](#head-22)
1. [Run export](#head-23)
1. [Provide trt engine generation specs](#head-26)
1. [Run TRT Engine generation using TAO-Deploy](#head-27)
1. [Provide TAO inference specs](#head-28)
1. [Run TAO inference](#head-29)
1. [Delete experiment](#head-30)
1. [Delete datasets](#head-31)
1. [Unmount shared volume](#head-32)
1. [Uninstall TAO Remote Client](#head-33)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import os
import glob
import subprocess
import getpass
import uuid
import json

In [ ]:
namespace = 'default'

### FIXME

1. Assign a model_name in FIXME 1
1. Assign the ip_address and port_number in FIXME 2 and FIXME 3 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
1. Assign the ngc_api_key variable in FIXME 4
1. (Optional) Enable AutoML if needed in FIXME 5
1. Choose between default and custom dataset in FIXME 6
1. Assign path of DATA_DIR in FIXME 7
1. Choose between Bayesian and Hyperband automl_algorithm in FIXME 8 (If automl was enabled in FIXME5)

In [ ]:
# Available models (#FIXME 1):
# 1. classification-pyt - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification.html
# 2. classification-tf1 - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification.html
# 3. classification-tf2 - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification_tf2.html
# 4. multitask-classification - https://docs.nvidia.com/tao/tao-toolkit/text/multitask_image_classification.html
# classification is the same as multi-class classification

model_name = "multitask-classification"  # FIXME1 (Add the model name from the above mentioned list)

### Install TAO remote client <a class="anchor" id="head-1"></a>

In [ ]:
# SKIP this step IF you have already installed the TAO-Client wheel.
! pip3 install nvidia-tao-client

In [ ]:
# View the version of the TAO-Client
! tao-client --version

### Set the remote service base URL <a class="anchor" id="head-2"></a>

In [ ]:
# Define the node_addr and port number
node_addr = "<ip_address>" # FIXME2 example: 10.137.149.22
node_port = "<port_number>" # FIXME3 example: 32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'

ngc_api_key = "<ngc_api_key>" # FIXME4 (Add NGC API key)

In [ ]:
automl_enabled = False # FIXME5 set to True if you want to run automl for the model chosen in the previous cell

In [ ]:
%env BASE_URL=http://{node_addr}:{node_port}/{namespace}/api/v1

# Exchange NGC_API_KEY for JWT
identity = json.loads(subprocess.getoutput(f'tao-client login --ngc-api-key {ngc_api_key}'))

%env USER={identity['user_id']}
%env TOKEN={identity['token']}

### Access the shared volume <a class="anchor" id="head-3"></a>

In [ ]:
# Get PVC ID
pvc_id = subprocess.getoutput(f'kubectl get pvc tao-toolkit-api-pvc -n {namespace} -o jsonpath="{{.spec.volumeName}}"')
print(pvc_id)

In [ ]:
# Get NFS server info
provisioner = json.loads(subprocess.getoutput(f'helm get values nfs-subdir-external-provisioner -o json'))
nfs_server = provisioner['nfs']['server']
nfs_path = provisioner['nfs']['path']
print(nfs_server, nfs_path)

In [ ]:
user = getpass.getuser()
home = os.path.expanduser('~')

! echo "Password for {user}"
password = getpass.getpass()

In [ ]:
# Mount shared volume 
! mkdir -p ~/shared

command = "apt-get -y install nfs-common >> /dev/null"
! echo {password} | sudo -S -k {command}

command = f"mount -t nfs {nfs_server}:{nfs_path}/{namespace}-tao-toolkit-api-pvc-{pvc_id} ~/shared"
! echo {password} | sudo -S -k {command} && echo DONE

### Create the datasets <a class="anchor" id="head-4"></a>

**For multi-class classification:**

We will be using the `pascal VOC dataset` for the tutorial. To find more details please visit [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#devkit). Please download the dataset present [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar) to the environment variable $DATA_DIR.

**If using custom dataset; it should follow this dataset structure, and skip running** "**Split dataset into train and val sets**"
```
DATA_DIR
├── classes.txt
├── images_test
│   ├── class_name_1
│   │   ├── image_name_1.jpg
│   │   ├── image_name_2.jpg
│   │   ├── ...
|   |   ... 
│   └── class_name_n
│       ├── image_name_3.jpg
│       ├── image_name_4.jpg
│       ├── ...
├── images_train
│   ├── class_name_1
│   │   ├── image_name_5.jpg
│   │   ├── image_name_6.jpg
|   |   ...
│   └── class_name_n
│       ├── image_name_7.jpg
│       ├── image_name_8.jpg
│       ├── ...
|
└── images_val
    ├── class_name_1
    │   ├── image_name_9.jpg
    │   ├── image_name_10.jpg
    │   ├── ...
    |   ...
    └── class_name_n
        ├── image_name_11.jpg
        ├── image_name_12.jpg
        ├── ...
```
- Each class name folder should contain the images corresponding to that class
- Same class name folders should be present across images_test, images_train and images_val
- classes.txt is a file which contains the names of all classes (each name in a separate line)

**For multi-task classification:**

We will be using the Fashion Product Images (Small) for the tutorial. This dataset is available on Kaggle.In this tutorial, our trained classification network will perform three tasks: article category classification, base color classification and target season classification.

To download the dataset, you will need a Kaggle account. After login, you can download the dataset zip file [here](https://www.kaggle.com/paramaggarwal/fashion-product-images-small). The downloaded file is archive.zip with a subfolder called myntradataset. Unzip contents in this subfolder to your workdir created in the cell above and you should have a folder called images and a CSV file called styles.csv

**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   |   ├── ...
├── styles.csv
```

In [ ]:
dataset_to_be_used = "default" # FIXME6 example: default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = model_name # FIXME7
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

In [ ]:
if dataset_to_be_used == "default":
    if "classification-" in model_name:
        if not os.path.exists(os.path.join(DATA_DIR,"VOCtrainval_11-May-2012.tar")):
            print("Download VOC tar data into ", DATA_DIR)
        else:
            !tar -xf $DATA_DIR/VOCtrainval_11-May-2012.tar -C $DATA_DIR
    elif model_name == "multitask-classification":
        if not os.path.exists(os.path.join(DATA_DIR,"archive.zip")):
            print(f"Download Fashion zip data into ", DATA_DIR)
        else:
            !unzip -uq $DATA_DIR/archive.zip -d $DATA_DIR/

In [ ]:
# Check the dataset is present
if "classification-" in model_name and dataset_to_be_used == "default":
    !if [ ! -d $DATA_DIR/VOCdevkit ]; then echo 'Images folder NOT found.'; else echo 'Found images folder.';fi
    !rm -rf $DATA_DIR/split
elif model_name == "multitask-classification":
    !if [ ! -d $DATA_DIR/images ]; then echo 'Images folder NOT found.'; else echo 'Found images folder.';fi
    !if [ ! -f $DATA_DIR/styles.csv ]; then echo 'CSV file NOT found.'; else echo 'Found CSV file.';fi
    # Create subdirectories and remove existing files in them
    !mkdir -p $DATA_DIR/images_train && rm -rf $DATA_DIR/images_train/*
    !mkdir -p $DATA_DIR/images_val && rm -rf $DATA_DIR/images_val/*
    !mkdir -p $DATA_DIR/images_test && rm -rf $DATA_DIR/images_test/*

### Split dataset into train and val sets

In [ ]:
# Split dataset into train and val sets
if "classification-" in model_name and dataset_to_be_used == "default":
    !python3 -m pip install tqdm
    from os.path import join as join_path
    import os
    import glob
    import re
    import shutil

    DATA_DIR=os.environ.get('DATA_DIR')
    source_dir = join_path(DATA_DIR, "VOCdevkit/VOC2012")
    target_dir = join_path(DATA_DIR, "formatted")

    suffix = '_trainval.txt'
    classes_dir = join_path(source_dir, "ImageSets", "Main")
    images_dir = join_path(source_dir, "JPEGImages")
    classes_files = glob.glob(classes_dir+"/*"+suffix)
    class_names = []
    for file in classes_files:
        # get the filename and make output class folder
        classname = os.path.basename(file)
        if classname.endswith(suffix):
            classname = classname[:-len(suffix)]
            target_dir_path = join_path(target_dir, classname)
            if not os.path.exists(target_dir_path):
                os.makedirs(target_dir_path)
        else:
            continue
        print(classname)
        class_names.append(classname)

        with open(file) as f:
            content = f.readlines()

        for line in content:
            tokens = re.split('\s+', line)
            if tokens[1] == '1':
                # copy this image into target dir_path
                target_file_path = join_path(target_dir_path, tokens[0] + '.jpg')
                src_file_path = join_path(images_dir, tokens[0] + '.jpg')
                shutil.copyfile(src_file_path, target_file_path)

    from random import shuffle
    from tqdm import tqdm

    DATA_DIR=os.environ.get('DATA_DIR')
    SOURCE_DIR=os.path.join(DATA_DIR, 'formatted')
    TARGET_DIR=os.path.join(DATA_DIR,'split')
    # list dir
    print(os.walk(SOURCE_DIR))
    dir_list = next(os.walk(SOURCE_DIR))[1]
    # for each dir, create a new dir in split
    for dir_i in tqdm(dir_list):
        newdir_train = os.path.join(TARGET_DIR, 'images_train', dir_i)
        newdir_val = os.path.join(TARGET_DIR, 'images_val', dir_i)
        newdir_test = os.path.join(TARGET_DIR, 'images_test', dir_i)

        if not os.path.exists(newdir_train):
                os.makedirs(newdir_train)
        if not os.path.exists(newdir_val):
                os.makedirs(newdir_val)
        if not os.path.exists(newdir_test):
                os.makedirs(newdir_test)

        img_list = glob.glob(os.path.join(SOURCE_DIR, dir_i, '*.jpg'))
        # shuffle data
        shuffle(img_list)

        for j in range(int(len(img_list)*0.7)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_train', dir_i))

        for j in range(int(len(img_list)*0.7), int(len(img_list)*0.8)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_val', dir_i))

        for j in range(int(len(img_list)*0.8), len(img_list)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_test', dir_i))

    with open(f'{DATA_DIR}/classes.txt', 'w') as f:
        for class_name in class_names:
            f.write(f"{class_name}\n")

    shutil.copy2(f'{DATA_DIR}/classes.txt',TARGET_DIR)
    print('Done splitting dataset.')

elif model_name == "multitask-classification" and dataset_to_be_used == "default":
    !python3 -m pip install numpy
    !python3 -m pip install pandas==1.5.1
    import os
    import shutil
    import numpy as np
    import pandas as pd

    df = pd.read_csv(os.environ['DATA_DIR'] + '/styles.csv', on_bad_lines='skip')
    df = df[['id', 'baseColour', 'subCategory', 'season']]
    df = df.dropna()
    category_cls = df.subCategory.value_counts()[:10].index # 10-class multitask-classification
    season_cls = ['Spring', 'Summer', 'Fall', 'Winter'] # 4-class multitask-classification
    color_cls = df.baseColour.value_counts()[:11].index # 11-class multitask-classification

    # Get all valid rows
    df = df[df.subCategory.isin(category_cls) & df.season.isin(season_cls) & df.baseColour.isin(color_cls)]
    df.columns = ['fname', 'base_color', 'category', 'season']
    df.fname = df.fname.astype(str)
    df.fname = df.fname + '.jpg'

    # remove entries whose image file is missing
    all_img_files = os.listdir(os.environ['DATA_DIR'] + '/images')
    df = df[df.fname.isin(all_img_files)]

    idx = np.arange(len(df))
    np.random.shuffle(idx)

    train_split_idx = int(len(df)*0.8)
    train_df = df.iloc[idx[:train_split_idx]]
    val_df = df.iloc[idx[train_split_idx:train_split_idx+(len(df) // 10)]]
    test_df = df.iloc[idx[train_split_idx+(len(df) // 10):]]

    # Add a simple sanity check
    assert len(train_df.season.unique()) == 4 and len(train_df.base_color.unique()) == 11 and \
        len(train_df.category.unique()) == 10, 'Training set misses some classes, re-run this cell!'
    assert len(val_df.season.unique()) == 4 and len(val_df.base_color.unique()) == 11 and \
        len(val_df.category.unique()) == 10, 'Validation set misses some classes, re-run this cell!'
    assert len(test_df.season.unique()) == 4 and len(test_df.base_color.unique()) == 11 and \
        len(test_df.category.unique()) == 10, 'Test set misses some classes, re-run this cell!'

    for image_name in train_df["fname"]:
        source_file_name = os.path.join(DATA_DIR, "images", image_name)
        destination_file_name = os.path.join(DATA_DIR, "images_train", image_name)
        shutil.copy(source_file_name, destination_file_name)

    for image_name in val_df["fname"]:
        source_file_name = os.path.join(DATA_DIR, "images", image_name)
        destination_file_name = os.path.join(DATA_DIR, "images_train", image_name)
        shutil.copy(source_file_name, destination_file_name)
        destination_file_name = os.path.join(DATA_DIR, "images_val", image_name)
        shutil.copy(source_file_name, destination_file_name)

    for image_name in test_df["fname"]:
        source_file_name = os.path.join(DATA_DIR, "images", image_name)
        destination_file_name = os.path.join(DATA_DIR, "images_test", image_name)
        shutil.copy(source_file_name, destination_file_name)

    # save processed data labels
    train_df.to_csv(os.environ['DATA_DIR'] + '/train.csv', index=False)
    val_df.to_csv(os.environ['DATA_DIR'] + '/val.csv', index=False)

### Verify the dataset split

In [ ]:
# verify
if "classification-" in model_name:
    !if [ ! -d $DATA_DIR/split/images_train ]; then echo 'train folder NOT found.'; else echo 'Found train images folder.';fi
    !if [ ! -d $DATA_DIR/split/images_val ]; then echo 'val folder NOT found.'; else echo 'Found val images folder.';fi
    !if [ ! -d $DATA_DIR/split/images_test ]; then echo 'test folder NOT found.'; else echo 'Found test images folder.';fi
    !if [ ! -f $DATA_DIR/split/classes.txt ]; then echo 'classes.txt not found'; else echo 'Found classes.txt.';fi
elif model_name == "multitask-classification":
    import pandas as pd

    print("Number of images in the train set. {}".format(
        len(pd.read_csv(os.environ['DATA_DIR'] + '/train.csv'))
    ))
    print("Number of images in the validation set. {}".format(
        len(pd.read_csv(os.environ['DATA_DIR'] + '/val.csv'))
    ))

### Create and upload datasets

In [ ]:
if model_name == "classification-pyt":
    ds_format = "classification_pyt"
elif "classification-" in model_name:
    ds_format = "default"
elif model_name == "multitask-classification":
    ds_format = "custom"

In [ ]:
train_dataset_id = subprocess.getoutput(f"tao-client {model_name} dataset-create --dataset_type image_classification --dataset_format {ds_format}")
print(train_dataset_id)

In [ ]:
if "classification-" in model_name:
    ! rsync -ah --info=progress2 {DATA_DIR}/split/images_train ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/
    ! rsync -ah --info=progress2 {DATA_DIR}/split/classes.txt ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/
elif model_name == "multitask-classification":
    ! rsync -ah --info=progress2 {DATA_DIR}/images_train ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/
    ! rsync -ah --info=progress2 {DATA_DIR}/train.csv ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/
    ! rsync -ah --info=progress2 {DATA_DIR}/val.csv ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/
! echo DONE

In [ ]:
eval_dataset_id = subprocess.getoutput(f"tao-client {model_name} dataset-create --dataset_type image_classification --dataset_format {ds_format}")
print(eval_dataset_id)

In [ ]:
if "classification-" in model_name:
    ! rsync -ah --info=progress2 {DATA_DIR}/split/images_val ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/
    ! rsync -ah --info=progress2 {DATA_DIR}/split/classes.txt ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/
elif model_name == "multitask-classification":
    ! rsync -ah --info=progress2 {DATA_DIR}/images_val ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}
    ! rsync -ah --info=progress2 {DATA_DIR}/val.csv ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/
! echo DONE

In [ ]:
test_dataset_id = subprocess.getoutput(f"tao-client {model_name} dataset-create --dataset_type image_classification --dataset_format raw")
print(test_dataset_id)

In [ ]:
if "classification-" in model_name:
    ! rsync -ah --info=progress2 {DATA_DIR}/split/images_test ~/shared/users/{os.environ['USER']}/datasets/{test_dataset_id}/
    ! rsync -ah --info=progress2 {DATA_DIR}/split/classes.txt ~/shared/users/{os.environ['USER']}/datasets/{test_dataset_id}/
elif model_name == "multitask-classification":
    ! rsync -ah --info=progress2 {DATA_DIR}/images_test ~/shared/users/{os.environ['USER']}/datasets/{test_dataset_id}
! echo DONE

### List datasets <a class="anchor" id="head-5"></a>

In [ ]:
pattern = os.path.join(home, 'shared', 'users', os.environ['USER'], 'datasets', '*', 'metadata.json')

datasets = []
for metadata_path in glob.glob(pattern):
    with open(metadata_path, 'r') as metadata_file:
        datasets.append(json.load(metadata_file))

print(json.dumps(datasets, indent=2))

### Create a model experiment <a class="anchor" id="head-6"></a>

In [ ]:
network_arch = model_name.replace("-","_")
if "classification_" in network_arch:
    encode_key = "nvidia_tlt"
else:
    encode_key = "tlt_encode"
model_id = subprocess.getoutput(f"tao-client {model_name} model-create --network_arch {network_arch} --encryption_key {encode_key} ")
print(model_id)

### Find pretrained model <a class="anchor" id="head-7"></a>

In [ ]:
# List all pretrained models for the chosen network architecture
pattern = os.path.join(home, 'shared', 'users', '*', 'models', '*', 'metadata.json')

for ptm_metadata_path in glob.glob(pattern):
  with open(ptm_metadata_path, 'r') as metadata_file:
    ptm_metadata = json.load(metadata_file)
    metadata_network_arch = ptm_metadata.get("network_arch")
    if metadata_network_arch == network_arch:
      if "encryption_key" not in ptm_metadata.keys():
        print(f'PTM Name: {ptm_metadata["name"]}; PTM version: {ptm_metadata["version"]}; NGC PATH: {ptm_metadata["ngc_path"]}; Additional info: {ptm_metadata["additional_id_info"]}')

In [ ]:
# Assigning pretrained models to different classification models
# From the output of previous cell make the appropriate changes to this map if you want to change the default PTM backbone.
# Changing the default backbone here requires changing default spec/config during train/eval etc like for example
# If you are changing the ptm to resnet34, then you have to modify the config key num_layers if it exists to 34 manually
pretrained_map = {"classification_tf1" : "pretrained_classification:resnet18",
                  "classification_tf2" : "pretrained_classification_tf2:efficientnet_b0",
                  "classification_pyt" : "pretrained_fan_classification_imagenet:fan_hybrid_tiny",
                  "multitask_classification" : "pretrained_classification:resnet10"}
no_ptm_models = set([])

In [ ]:
if network_arch not in no_ptm_models:
    pattern = os.path.join(home, 'shared', 'users', '*', 'models', '*', 'metadata.json')

    ptm = []
    for ptm_metadata_path in glob.glob(pattern):
      with open(ptm_metadata_path, 'r') as metadata_file:
        ptm_metadata = json.load(metadata_file)
        ngc_path = ptm_metadata.get("ngc_path")
        metadata_network_arch = ptm_metadata.get("network_arch")
        if metadata_network_arch == network_arch and ngc_path.endswith(pretrained_map[network_arch]):
          ptm = [ptm_metadata["id"]]
          break

    print(ptm)

### Customize model metadata <a class="anchor" id="head-10"></a>

In [ ]:
metadata_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'metadata.json')

with open(metadata_path , "r") as metadata_file:
    metadata = json.load(metadata_file)

metadata["train_datasets"] = [train_dataset_id]
metadata["eval_dataset"] = eval_dataset_id
metadata["inference_dataset"] = test_dataset_id
metadata["calibration_dataset"] = train_dataset_id
if network_arch not in no_ptm_models:
    metadata["ptm"] = ptm

with open(metadata_path, "w") as metadata_file:
    json.dump(metadata, metadata_file, indent=2)

print(json.dumps(metadata, indent=2))

### View hyperparameters that are enabled for AutoML by default <a class="anchor" id="head-11"></a>

In [ ]:
if automl_enabled:
    # View default automl specs enabled
    ! tao-client {model_name} model-automl-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/automl_defaults.json

### Set AutoML related configurations <a class="anchor" id="head-12"></a>
Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters: 

[Classification TF1](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_tf1/classification_tf1%20-%20train.csv), 
[Classification TF2](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_tf2/classification_tf2%20-%20train.csv), 
[Classification Pytorch](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/classification_pyt/classification_pyt%20-%20train.csv), 
[Multitask classification](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/multitask_classification/multitask_classification%20-%20train.csv)

In [ ]:
if automl_enabled:
    # Choose automl algorithm between "Bayesian" and "HyperBand".
    automl_algorithm="Bayesian" # FIXME8 example: Bayesian/HyperBand

    if model_name == "classification-pyt":
        metric = "loss"
    else:
        metric = "kpi" 

    additional_automl_parameters = [] #Refer to parameter list mentioned in the above links and add any extra parameter in addition to the default enabled ones
    remove_default_automl_parameters = [] #Remove any hyperparameters that are enabled by default for AutoML

    metadata["automl_algorithm"] = automl_algorithm
    metadata["automl_enabled"] = automl_enabled
    metadata["metric"] = metric
    metadata["epoch_multiplier"] = 1 # Will be considered for Hyperband only
    metadata["automl_add_hyperparameters"] = str(additional_automl_parameters)
    metadata["automl_remove_hyperparameters"] = str(remove_default_automl_parameters)

    with open(metadata_path, "w") as metadata_file:
        json.dump(metadata, metadata_file, indent=2)

    print(json.dumps(metadata, indent=2))

### Provide train specs <a class="anchor" id="head-13"></a>

In [ ]:
# Default train model specs
! tao-client {model_name} model-train-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/train.json

In [ ]:
# Customize train model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'train.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

# Example for multitask-classification (for each network the parameter key might be different)
if model_name == "multitask-classification":
    specs["training_config"]["num_epochs"] = 10
    specs["gpus"] = 1
# Example for classification-pyt
elif model_name == "classification-pyt":
    specs["train"]["train_config"]["runner"]["max_epochs"] = 40
    specs["train"]["num_gpus"] = 1
    specs["gpus"] = 1
# Example for classification-tf1
elif model_name == "classification-tf1":
    specs["train_config"]["n_epochs"] = 80
    specs["gpus"] = 1
# Example for classification-tf2
elif model_name == "classification-tf2":
    specs["train"]["num_epochs"] = 80
    specs["gpus"] = 1

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run train <a class="anchor" id="head-14"></a>

In [ ]:
train_job_id = subprocess.getoutput(f"tao-client {model_name} model-train --id " + model_id)
print(train_job_id)

In [ ]:
def my_tail(logs_dir, log_file):
    %env LOG_FILE={logs_dir}/{log_file}
    ! mkdir -p {logs_dir}
    ! [ ! -f "$LOG_FILE" ] && touch $LOG_FILE && chmod 666 $LOG_FILE
    ! tail -f -n +1 $LOG_FILE | while read LINE; do echo "$LINE"; [[ "$LINE" == "EOF" ]] && pkill -P $$ tail; done

In [ ]:
# Monitor job status
if automl_enabled:
    # Set poll_automl_stats to True if just want to see what's the time left, how many epochs are remaining etc.
    # Set poll_automl_stats to False if you want to skip stats and see the training logs instead. Training logs viewing are supported only for Bayesian

    # For automl: Training times for different models benchmarked on 1 GPU V100 machine can be found here: https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#results-of-automl-experiments
    
    poll_automl_stats = True
    if poll_automl_stats:
        import time
        from IPython.display import clear_output
        stats_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, train_job_id, "automl_metadata.json")
        controller_json_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, train_job_id, "controller.json")
        while True:
            time.sleep(15)
            clear_output(wait=True)
            if os.path.exists(stats_path):
                try:
                    with open(stats_path , "r") as stats_file:
                        stats_dict = json.load(stats_file)
                    print(json.dumps(stats_dict, indent=2))
                    if float(stats_dict.get("Number of epochs yet to start",-1)) == 0.0 or float(stats_dict.get("Number of iters yet to start",-1)) == 0.0:
                        break
                except (json.JSONDecodeError):
                    print("Stats computed are being written to file. Stats will be visible on screen in a few seconds")
    else:
        # Print the log file - supported only for bayesian (the file won't exist until the backend Toolkit container is running -- can take several minutes)
        if automl_algorithm == "Bayesian":
            logs_dir = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id)
            max_recommendations = metadata.get("automl_max_recommendations",20)
            for experiment_num in range(max_recommendations):
                log_file = f"{train_job_id}/experiment_{experiment_num}/log.txt"
                while True:
                    if os.path.exists(os.path.join(logs_dir, log_file)):
                        break
                print(f"\n\nViewing experiment {experiment_num}\n\n")
                my_tail(logs_dir, log_file)
    
else:
    # Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
    logs_dir = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'logs')
    log_file = f"{train_job_id}.txt"

    my_tail(logs_dir, log_file)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor job status' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# if automl_enabled:
#     canceled_job_id = subprocess.getoutput(f"tao-client {model_name} model-job-cancel --id {model_id} --job {train_job_id}")
#     print(canceled_job_id)

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor job status' cell above (4th cell above from this cell)
# if automl_enabled:
#     resumed_job_id = subprocess.getoutput(f"tao-client {model_name} model-job-resume --id {model_id} --job {train_job_id}")
#     print(resumed_job_id)

### Viewing checkpoint files <a class="anchor" id="head-15"></a>

In [ ]:
# View the checkpoints generated for the training job and for automl jobs, in addition view: best performing model's config and the results of all automl experiments

job_dir = f"{home}/shared/users/{os.environ['USER']}/models/{model_id}/{train_job_id}"
model_path = job_dir

if automl_enabled:
    !python3 -m pip install pandas==1.5.1
    import pandas as pd
    import glob
    model_path =  f"{job_dir}/best_model"

from IPython.display import clear_output

while True:
    clear_output(wait=True)
    if os.path.exists(model_path) and len(os.listdir(model_path)) > 0:
        #List the binary model file
        print("\nCheckpoints for the training experiment")
        if os.path.exists(model_path+"/train/weights") and len(os.listdir(model_path+"/train/weights")) > 0:
            print(f"Folder: {model_path}/train/weights")
            print("Files:", os.listdir(model_path+"/train/weights"))
        elif os.path.exists(model_path+"/weights") and len(os.listdir(model_path+"/weights")) > 0:
            print(f"Folder: {model_path}/weights")
            print("Files:", os.listdir(model_path+"/weights"))
        else:
            print(f"Folder: {model_path}")
            print("Files:", os.listdir(model_path))

        if automl_enabled:
            if os.path.exists(f"{model_path}/controller.json") and (len(glob.glob(os.path.join(model_path,"*.protobuf"))) > 0 or len(glob.glob(os.path.join(model_path,"*.yaml"))) > 0):
                experiment_artifacts = json.load(open(f"{model_path}/controller.json","r"))
                data_frame = pd.DataFrame(experiment_artifacts)
                # Print experiment id/number and the corresponding result
                print("\nResults of all experiments")
                with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
                    print(data_frame[["id","result"]])
                break
        else:
            break

### Provide evaluate specs <a class="anchor" id="head-16"></a>

In [ ]:
# Default evaluate model specs
! tao-client {model_name} model-evaluate-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/evaluate.json

In [ ]:
# Customize evaluate model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'evaluate.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

# Change any spec if you wish

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run evaluate <a class="anchor" id="head-17"></a>

In [ ]:
eval_job_id = subprocess.getoutput(f"tao-client {model_name} model-evaluate --id {model_id} --job {train_job_id}")
print(eval_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{eval_job_id}.txt"
logs_dir = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'logs')
my_tail(logs_dir, log_file)

### Provide prune specs <a class="anchor" id="head-18"></a>

In [ ]:
if model_name != "classification-pyt":
    # Default prune model specs
    ! tao-client {model_name} model-prune-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/prune.json

In [ ]:
if model_name != "classification-pyt":
    # Customize retrain model specs
    specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'prune.json')

    with open(specs_path , "r") as specs_file:
        specs = json.load(specs_file)

    # Apply changes to specs dictionary if required here
    if model_name == "classification-tf2":
        specs["prune"]["byom_model_path"] = ""

    with open(specs_path, "w") as specs_file:
        json.dump(specs, specs_file, indent=2)

    print(json.dumps(specs, indent=2))

### Run prune <a class="anchor" id="head-19"></a>

In [ ]:
if model_name != "classification-pyt":
    prune_job_id = subprocess.getoutput(f"tao-client {model_name} model-prune --id {model_id} --job {train_job_id}")
    print(prune_job_id)

In [ ]:
if model_name != "classification-pyt":
    # Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
    log_file = f"{prune_job_id}.txt"
    my_tail(logs_dir, log_file)

### Provide retrain specs <a class="anchor" id="head-20"></a>

In [ ]:
if model_name != "classification-pyt":
    # Default retrain model specs
    ! tao-client {model_name} model-retrain-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/retrain.json

In [ ]:
if model_name != "classification-pyt":
    # Customize retrain model specs
    specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'retrain.json')

    with open(specs_path , "r") as specs_file:
        specs = json.load(specs_file)

    # Override any of the parameters listed in the previous cell as required
    # Example for multitask-classification (for each network the parameter key might be different)
    if model_name == "multitask-classification":
        specs["training_config"]["num_epochs"] = 10
        specs["gpus"] = 1
    # Example for classification_tf1
    elif model_name == "classification-tf1":
        specs["train_config"]["n_epochs"] = 80
        specs["gpus"] = 1
    # Example for classification_tf2
    elif model_name == "classification-tf2":
        specs["train"]["num_epochs"] = 80
        specs["gpus"] = 1

    with open(specs_path, "w") as specs_file:
        json.dump(specs, specs_file, indent=2)

    print(json.dumps(specs, indent=2))

### Run retrain <a class="anchor" id="head-21"></a>

In [ ]:
if model_name != "classification-pyt":
    retrain_job_id = subprocess.getoutput(f"tao-client {model_name} model-retrain --id {model_id} --job {prune_job_id}")
    print(retrain_job_id)

In [ ]:
if model_name != "classification-pyt":
    # Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
    log_file = f"{retrain_job_id}.txt"
    my_tail(logs_dir, log_file)

### Run evaluate on retrained model <a class="anchor" id="head-21-1"></a>

In [ ]:
if model_name != "classification-pyt":
    eval2_job_id = subprocess.getoutput(f"tao-client {model_name} model-evaluate --id {model_id} --job {retrain_job_id}")
    print(eval2_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
if model_name != "classification-pyt":
    log_file = f"{eval2_job_id}.txt"
    my_tail(logs_dir, log_file)

### Provide export specs <a class="anchor" id="head-22"></a>

In [ ]:
# Default export model specs
! tao-client {model_name} model-export-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/export.json

In [ ]:
# Customize export model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'export.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

#Apply changes to the specs dictionary here if required

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run export <a class="anchor" id="head-23"></a>

In [ ]:
export_job_id = subprocess.getoutput(f"tao-client {model_name} model-export --id {model_id} --job {train_job_id}")
print(export_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{export_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide trt engine generation specs <a class="anchor" id="head-26"></a>

In [ ]:
# Default gen_trt_engine model specs
! tao-client {model_name} model-gen-trt-engine-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/gen_trt_engine.json

In [ ]:
# Customize gen_trt_engine model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'gen_trt_engine.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

#Apply changes to the specs dictionary here if required
if model_name == "classification-tf2":
    specs["gen_trt_engine"]["tensorrt"]["data_type"] = "fp16"
elif model_name == "classification-pyt":
    specs["gen_trt_engine"]["tensorrt"]["data_type"] = "fp16"
else:
    specs["data_type"] = "int8"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run TRT Engine generation using TAO-Deploy <a class="anchor" id="head-27"></a>

In [ ]:
gen_trt_engine_job_id = subprocess.getoutput(f"tao-client {model_name} model-gen-trt-engine --id {model_id} --job {export_job_id}")
print(gen_trt_engine_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{gen_trt_engine_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide TAO inference specs <a class="anchor" id="head-28"></a>

In [ ]:
# Default inference model specs
! tao-client {model_name} model-inference-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/inference.json

In [ ]:
# Customize TAO inference specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'inference.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

#Apply changes to the specs dictionary here if required

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run TAO inference <a class="anchor" id="head-29"></a>

In [ ]:
tlt_inference_job_id = subprocess.getoutput(f"tao-client {model_name} model-inference --id {model_id} --job {train_job_id}")
print(tlt_inference_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{tlt_inference_job_id}.txt"
my_tail(logs_dir, log_file)

In [ ]:
job_dir = f"{home}/shared/users/{os.environ['USER']}/models/{model_id}/{tlt_inference_job_id}"
# You can find the predicted results here
!ls {job_dir}

### Provide TRT inference specs <a class="anchor" id="head-30"></a>

In [ ]:
# Default inference model specs
! tao-client {model_name} model-inference-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/inference.json

In [ ]:
# Customize TAO inference specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'inference.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

#Apply changes to the specs dictionary here if required

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run TRT inference <a class="anchor" id="head-31"></a>

In [ ]:
trt_inference_job_id = subprocess.getoutput(f"tao-client {model_name} model-inference --id {model_id} --job {gen_trt_engine_job_id}")
print(trt_inference_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{trt_inference_job_id}.txt"
my_tail(logs_dir, log_file)

In [ ]:
job_dir = f"{home}/shared/users/{os.environ['USER']}/models/{model_id}/{tlt_inference_job_id}"
# You can find the predicted results here
!ls {job_dir}

### Delete experiment <a class="anchor" id="head-30"></a>

In [ ]:
! rm -rf ~/shared/users/{os.environ['USER']}/models/{model_id}
! echo DONE

### Delete datasets <a class="anchor" id="head-31"></a>

In [ ]:
! rm -rf ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}
! rm -rf ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}
! rm -rf ~/shared/users/{os.environ['USER']}/datasets/{infer_dataset_id}
! echo DONE

### Unmount shared volume <a class="anchor" id="head-32"></a>

In [ ]:
command = "umount ~/shared"
! echo {password} | sudo -S -k {command} && echo DONE

### Uninstall TAO Remote Client <a class="anchor" id="head-33"></a>

In [ ]:
! pip3 uninstall -y nvidia-tao-client